In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import credentials
import requests
#pulling data API
key =credentials.login['key_isaham']  # extract password

In [10]:
# Define the instruments to download.

tech = ["GREATEC",
        "UWC",       
        "D&O", 
       "GENETEC", 
       "UNISEM", 
       "FRONTKN", 
       "ATAIMS", 
       "FPI", 
       "PENTA", 
       "DUFU", 
       "VS", 
       "SKPRES", 
        "MPI", 
        "VITROX"
       ]   
       


for m,stock in enumerate (tech):
    print("----------------PROCESSING {}------------------".format(stock))
    #symbol = "UNISEM"  # which stock to pull
    symbol = stock    
    
    response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
    data = response.json()
    #print(data)
    plt.style.use('fivethirtyeight')

    %config InlineBackend.figure_format = 'retina'
    %matplotlib inline

    # makes plots bigger
    plt.rcParams["figure.figsize"] = (20,8)
    
    #Tuning OB and OS level
    ob_cross=85
    os_cross=35

    df= pd.DataFrame.from_dict(data)
    df= df[['c','d','h','l','o','v']]
    df['d'] = pd.to_datetime(df['d'], dayfirst= True )
    #identify MA50
    n=50
    ma50=[] 

    for i in range(len(df["c"])-(n-1)):
        ma = round(sum(df["c"][i:i+(n)])/n,4)
        #print(ma)
        ma50.append(ma)
    
    df["MA50"] = pd.Series(ma50)
    df["MA50"] = df["MA50"].shift(n-1)

    #filter uptrend price above ma50
    # Calculating the gap of both MA
    df["MAgap"] = df["c"] -  df["MA50"] 

    df.set_index('d',inplace=True,drop=True)
    df.index.name = None

    #create signal for uptrend price above ma50 
    #result not accurate for stochastic when just filter uptrend only
    df["Uptrend"]=df['MAgap']>0
    Uptrend=df["Uptrend"]
    Uptrend_signal=Uptrend[Uptrend==True]
    #signal stochastic
    #Create the "L14" column in the DataFrame
    df['L14'] = df['l'].rolling(window=14).min()

    #Create the "H14" column in the DataFrame
    df['H14'] = df['h'].rolling(window=14).max()

    #Create the "%K" column in the DataFrame
    df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

    #Create the "%D" column in the DataFrame
    df['%D'] = df['%K'].rolling(window=3).mean()

    #Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
    #A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
    df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

    #Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
    #A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
    df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 

    overbought =df['Sell OB']
    oversold =df['Buy OS']
    date_os=list(oversold[oversold == True].index)
    date_ob=list(overbought[overbought == True].index)
    
    # transaction recording
    buy = []
    sell = []
    sdate = []
    bdate = []
    wr = []
    pct = []
    
    for n,i in enumerate(date_os):
        #print(n,"Buy:", i, "Price:", df['c'].loc[i])
        buy.append(df['c'].loc[i])
        bdate.append(i)
        r=[]
        #print("before", len(r))
        for m,k in enumerate(date_ob):
            
            if k>i:
                r.append(m)
                #print('after',len(r))
                if len(r) ==1:
                    #print(m,"Sell :", k, "Price:",df['c'].loc[k])
                    #print("Percentage change:",(((df['c'].loc[k]/df['c'].loc[i])-1)*100) )
                    sell.append(df['c'].loc[k])
                    sdate.append(k)
                    pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                    pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                elif len(r)>1:
                    pass
    diff =len(buy)-len(sell)
    # Checking for final transaction, make sure it is sell, as we want to close the position
    if len(buy) == len(sell):
        #print("Yes \n", "Buy:", len(buy), "Sell:", len(sell))
        #print("Position closed")
        pass
    
    elif len(buy) != len(sell):
        #print("No, \n", "Buy:{}, Sell:{}, Closing the position".format(len(buy),len(sell)))
        del(buy[-diff:])
        del(bdate[-diff:])
        #print("Position closed")
    dct = {
    "Buy_Date": bdate,
    "Buy": buy,
    "Sell": sell,
    "Sell_Date": sdate,
    "Pct_change": pct }   
    trans = pd.DataFrame(dct)
    wr = []
    for i in range(len(trans)):
        if trans["Pct_change"][i] > 0:
            wr.append(1)
        else:
            wr.append(0)
    trans["wr"] = pd.Series(wr)
    
    Name_stock = []
    for s in range(len(trans)):
        Name_stock.append(symbol)
        
    #X_train, y_train = X['01-01-2016': '01-01-2021'], y['01-01-2016': '01-01-2021']
    trans['Stock'] = pd.Series(Name_stock)
    #trans['01-01-2020': '01-01-2021']
    
    trans.set_index('Buy_Date', inplace=True)
    
    trans=trans['01-01-2020':'01-01-2021']
    print(trans)
    net_pct=round(sum(trans['Pct_change']),4)
    wrate = round(len(trans[trans["wr"]==1])/len(trans["wr"])*100, 4)
    print('Stock :{}'.format(symbol))
    print('When Overbought cross>{}.'.format(ob_cross))
    print('When Oversold cross<{}.'.format(os_cross))
    print("Winning rate at {}% for {} trades and winning percentage {}%.".format(wrate, len(trans["wr"]),net_pct))
    risk = round((100-wrate),2)
    print("While, using this strategy, we are exposed to {}% of lossing risk for {} trades.".format(risk, len(trans["wr"])))
    print("----------------DONE {}------------------".format(symbol))
    

print("Finish")

    



----------------PROCESSING GREATEC------------------
              Buy  Sell  Sell_Date  Pct_change  wr    Stock
Buy_Date                                                   
2020-03-10  1.500  1.29 2020-04-15  -14.000000   0  GREATEC
2020-09-11  3.000  3.34 2020-10-02   11.333333   1  GREATEC
2020-09-18  3.075  3.34 2020-10-02    8.617886   1  GREATEC
2020-09-23  3.010  3.34 2020-10-02   10.963455   1  GREATEC
2020-12-07  4.250  4.65 2020-12-28    9.411765   1  GREATEC
2020-12-15  4.130  4.65 2020-12-28   12.590799   1  GREATEC
2020-12-18  4.165  4.65 2020-12-28   11.644658   1  GREATEC
Stock :GREATEC
When Overbought cross>85.
When Oversold cross<35.
Winning rate at 85.7143% for 7 trades and winning percentage 50.5619%.
While, using this strategy, we are exposed to 14.29% of lossing risk for 7 trades.
----------------DONE GREATEC------------------
----------------PROCESSING UWC------------------
              Buy   Sell  Sell_Date  Pct_change  wr Stock
Buy_Date                          

             Buy   Sell  Sell_Date  Pct_change  wr Stock
Buy_Date                                                
2020-02-03  1.40  0.925 2020-04-13  -33.928571   0    VS
2020-06-02  0.91  1.390 2020-07-29   52.747253   1    VS
2020-06-09  0.92  1.390 2020-07-29   51.086957   1    VS
2020-07-01  0.98  1.390 2020-07-29   41.836735   1    VS
2020-07-06  1.03  1.390 2020-07-29   34.951456   1    VS
2020-10-30  2.34  2.700 2020-12-04   15.384615   1    VS
2020-11-03  2.35  2.700 2020-12-04   14.893617   1    VS
2020-11-26  2.35  2.700 2020-12-04   14.893617   1    VS
2020-12-23  2.58  2.720 2021-01-21    5.426357   1    VS
Stock :VS
When Overbought cross>85.
When Oversold cross<35.
Winning rate at 88.8889% for 9 trades and winning percentage 197.292%.
While, using this strategy, we are exposed to 11.11% of lossing risk for 9 trades.
----------------DONE VS------------------
----------------PROCESSING SKPRES------------------
             Buy  Sell  Sell_Date  Pct_change  wr   Stock
Buy_Dat

In [12]:
total_wr =85.71 + 60 + 100 + 91.67 + 100 + 85.71 + 100 + 100 + 83.33 + 66.67 + 88.89 + 83.33 + 100 + 50

In [16]:
total_wr

1195.31

In [15]:
round(total_wr/14,2)

85.38

In [19]:
total_wp =50.56 + (-5.78) + 214.33 + 200.48 + 93.34 + 112.49 + 109.50 + 112.49 + 28.92 + 49.60 + 197.29 + 133.87 + 6.16 + 4.87
total_wp

1308.1200000000001

In [18]:
round(total_wp/14,2)

93.44

### Trading plan 2019

In [20]:
# Define the instruments to download.

tech = ["GREATEC",
        "UWC",       
        "D&O", 
       "GENETEC", 
       "UNISEM", 
       "FRONTKN", 
       "ATAIMS", 
       "FPI", 
       "PENTA", 
       "DUFU", 
       "VS", 
       "SKPRES", 
        "MPI", 
        "VITROX"
       ]   
       


for m,stock in enumerate (tech):
    print("----------------PROCESSING {}------------------".format(stock))
    #symbol = "UNISEM"  # which stock to pull
    symbol = stock    
    
    response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
    data = response.json()
    #print(data)
    plt.style.use('fivethirtyeight')

    %config InlineBackend.figure_format = 'retina'
    %matplotlib inline

    # makes plots bigger
    plt.rcParams["figure.figsize"] = (20,8)
    
    #Tuning OB and OS level
    ob_cross=85
    os_cross=35

    df= pd.DataFrame.from_dict(data)
    df= df[['c','d','h','l','o','v']]
    df['d'] = pd.to_datetime(df['d'], dayfirst= True )
    #identify MA50
    n=50
    ma50=[] 

    for i in range(len(df["c"])-(n-1)):
        ma = round(sum(df["c"][i:i+(n)])/n,4)
        #print(ma)
        ma50.append(ma)
    
    df["MA50"] = pd.Series(ma50)
    df["MA50"] = df["MA50"].shift(n-1)

    #filter uptrend price above ma50
    # Calculating the gap of both MA
    df["MAgap"] = df["c"] -  df["MA50"] 

    df.set_index('d',inplace=True,drop=True)
    df.index.name = None

    #create signal for uptrend price above ma50 
    #result not accurate for stochastic when just filter uptrend only
    df["Uptrend"]=df['MAgap']>0
    Uptrend=df["Uptrend"]
    Uptrend_signal=Uptrend[Uptrend==True]
    #signal stochastic
    #Create the "L14" column in the DataFrame
    df['L14'] = df['l'].rolling(window=14).min()

    #Create the "H14" column in the DataFrame
    df['H14'] = df['h'].rolling(window=14).max()

    #Create the "%K" column in the DataFrame
    df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

    #Create the "%D" column in the DataFrame
    df['%D'] = df['%K'].rolling(window=3).mean()

    #Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
    #A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
    df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

    #Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
    #A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
    df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 

    overbought =df['Sell OB']
    oversold =df['Buy OS']
    date_os=list(oversold[oversold == True].index)
    date_ob=list(overbought[overbought == True].index)
    
    # transaction recording
    buy = []
    sell = []
    sdate = []
    bdate = []
    wr = []
    pct = []
    
    for n,i in enumerate(date_os):
        #print(n,"Buy:", i, "Price:", df['c'].loc[i])
        buy.append(df['c'].loc[i])
        bdate.append(i)
        r=[]
        #print("before", len(r))
        for m,k in enumerate(date_ob):
            
            if k>i:
                r.append(m)
                #print('after',len(r))
                if len(r) ==1:
                    #print(m,"Sell :", k, "Price:",df['c'].loc[k])
                    #print("Percentage change:",(((df['c'].loc[k]/df['c'].loc[i])-1)*100) )
                    sell.append(df['c'].loc[k])
                    sdate.append(k)
                    pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                    pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                elif len(r)>1:
                    pass
    diff =len(buy)-len(sell)
    # Checking for final transaction, make sure it is sell, as we want to close the position
    if len(buy) == len(sell):
        #print("Yes \n", "Buy:", len(buy), "Sell:", len(sell))
        #print("Position closed")
        pass
    
    elif len(buy) != len(sell):
        #print("No, \n", "Buy:{}, Sell:{}, Closing the position".format(len(buy),len(sell)))
        del(buy[-diff:])
        del(bdate[-diff:])
        #print("Position closed")
    dct = {
    "Buy_Date": bdate,
    "Buy": buy,
    "Sell": sell,
    "Sell_Date": sdate,
    "Pct_change": pct }   
    trans = pd.DataFrame(dct)
    wr = []
    for i in range(len(trans)):
        if trans["Pct_change"][i] > 0:
            wr.append(1)
        else:
            wr.append(0)
    trans["wr"] = pd.Series(wr)
    
    Name_stock = []
    for s in range(len(trans)):
        Name_stock.append(symbol)
        
    #X_train, y_train = X['01-01-2016': '01-01-2021'], y['01-01-2016': '01-01-2021']
    trans['Stock'] = pd.Series(Name_stock)
    #trans['01-01-2020': '01-01-2021']
    
    trans.set_index('Buy_Date', inplace=True)
    
    trans=trans['01-01-2019':'01-01-2020']
    print(trans)
    net_pct=round(sum(trans['Pct_change']),4)
    wrate = round(len(trans[trans["wr"]==1])/len(trans["wr"])*100, 4)
    print('Stock :{}'.format(symbol))
    print('When Overbought cross>{}.'.format(ob_cross))
    print('When Oversold cross<{}.'.format(os_cross))
    print("Winning rate at {}% for {} trades and winning percentage {}%.".format(wrate, len(trans["wr"]),net_pct))
    risk = round((100-wrate),2)
    print("While, using this strategy, we are exposed to {}% of lossing risk for {} trades.".format(risk, len(trans["wr"])))
    print("----------------DONE {}------------------".format(symbol))
    

print("Finish")


----------------PROCESSING GREATEC------------------
              Buy   Sell  Sell_Date  Pct_change  wr    Stock
Buy_Date                                                    
2019-11-25  0.960  1.095 2019-12-09   14.062500   1  GREATEC
2019-11-27  0.935  1.095 2019-12-09   17.112299   1  GREATEC
Stock :GREATEC
When Overbought cross>85.
When Oversold cross<35.
Winning rate at 100.0% for 2 trades and winning percentage 31.1748%.
While, using this strategy, we are exposed to 0.0% of lossing risk for 2 trades.
----------------DONE GREATEC------------------
----------------PROCESSING UWC------------------
               Buy    Sell  Sell_Date  Pct_change  wr Stock
Buy_Date                                                   
2019-10-01  0.5233  0.5533 2019-10-08    5.732849   1   UWC
2019-12-05  0.8700  1.0467 2019-12-26   20.310345   1   UWC
Stock :UWC
When Overbought cross>85.
When Oversold cross<35.
Winning rate at 100.0% for 2 trades and winning percentage 26.0432%.
While, using this stra

             Buy  Sell  Sell_Date  Pct_change  wr   Stock
Buy_Date                                                 
2019-04-04  7.10  7.25 2019-04-25    2.112676   1  VITROX
2019-04-11  7.11  7.25 2019-04-25    1.969058   1  VITROX
2019-04-18  7.12  7.25 2019-04-25    1.825843   1  VITROX
Stock :VITROX
When Overbought cross>85.
When Oversold cross<35.
Winning rate at 100.0% for 3 trades and winning percentage 5.9076%.
While, using this strategy, we are exposed to 0.0% of lossing risk for 3 trades.
----------------DONE VITROX------------------
Finish


In [21]:
total_wr_19 =100 + 100 + 50 + 0 + 100 + 100 + 100 + 20 + 75 + 100 + 83.33 + 85.71 + 80 + 100 
total_wr_19

1094.04

In [22]:
round(total_wr_19/14,2)

78.15

In [24]:
total_wp_19 =31.17+ 26.04 + (-9.62) + (-84.74) + 15.43 + 148.20 + 14.41 + 4.93 + 30.11 + 25.84 + 40.48 + 24.63 + 8.47 + 5.91
total_wp_19

281.26000000000005

In [25]:
round(total_wp_19/14,2)

20.09

### Trading plan 2018

In [30]:
# Define the instruments to download.
# Exclude UWC and GREATEC (IPO 2019)
tech = [
           
        "D&O", 
       "GENETEC", 
       "UNISEM", 
       "FRONTKN", 
       "ATAIMS", 
       "FPI", 
       "PENTA", 
       "DUFU", 
       "VS", 
       "SKPRES", 
        "MPI", 
        "VITROX"
       ]   
       


for m,stock in enumerate (tech):
    print("----------------PROCESSING {}------------------".format(stock))
    #symbol = "UNISEM"  # which stock to pull
    symbol = stock    
    
    response = requests.get("https://admin.isaham.my/api/chart?stock={}&key={}".format(symbol,key))
    data = response.json()
    #print(data)
    plt.style.use('fivethirtyeight')

    %config InlineBackend.figure_format = 'retina'
    %matplotlib inline

    # makes plots bigger
    plt.rcParams["figure.figsize"] = (20,8)
    
    #Tuning OB and OS level
    ob_cross=85
    os_cross=35

    df= pd.DataFrame.from_dict(data)
    df= df[['c','d','h','l','o','v']]
    df['d'] = pd.to_datetime(df['d'], dayfirst= True )
    #identify MA50
    n=50
    ma50=[] 

    for i in range(len(df["c"])-(n-1)):
        ma = round(sum(df["c"][i:i+(n)])/n,4)
        #print(ma)
        ma50.append(ma)
    
    df["MA50"] = pd.Series(ma50)
    df["MA50"] = df["MA50"].shift(n-1)

    #filter uptrend price above ma50
    # Calculating the gap of both MA
    df["MAgap"] = df["c"] -  df["MA50"] 

    df.set_index('d',inplace=True,drop=True)
    df.index.name = None

    #create signal for uptrend price above ma50 
    #result not accurate for stochastic when just filter uptrend only
    df["Uptrend"]=df['MAgap']>0
    Uptrend=df["Uptrend"]
    Uptrend_signal=Uptrend[Uptrend==True]
    #signal stochastic
    #Create the "L14" column in the DataFrame
    df['L14'] = df['l'].rolling(window=14).min()

    #Create the "H14" column in the DataFrame
    df['H14'] = df['h'].rolling(window=14).max()

    #Create the "%K" column in the DataFrame
    df['%K'] = 100*((df['c'] - df['L14']) / (df['H14'] - df['L14']) )

    #Create the "%D" column in the DataFrame
    df['%D'] = df['%K'].rolling(window=3).mean()

    #Create a column in the DataFrame showing "TRUE" if buy entry signal is given and "FALSE" otherwise. 
    #A buy is initiated when the %K line crosses up through the %D line and the value of the oscillator is below 20 
    df['Buy OS'] = ((df['%K'] > df['%D']) & (df['%K'].shift(1) < df['%D'].shift(1))) & (df['%D'] < os_cross) & (Uptrend_signal)

    #Create a column in the DataFrame showing "TRUE" if sell entry signal is given and "FALSE" otherwise. 
    #A sell is initiated when the %K line crosses down through the %D line and the value of the oscillator is above 80 
    df['Sell OB'] = ((df['%K'] < df['%D']) & (df['%K'].shift(1) > df['%D'].shift(1))) & (df['%D'] > ob_cross) 

    overbought =df['Sell OB']
    oversold =df['Buy OS']
    date_os=list(oversold[oversold == True].index)
    date_ob=list(overbought[overbought == True].index)
    
    # transaction recording
    buy = []
    sell = []
    sdate = []
    bdate = []
    wr = []
    pct = []
    
    for n,i in enumerate(date_os):
        #print(n,"Buy:", i, "Price:", df['c'].loc[i])
        buy.append(df['c'].loc[i])
        bdate.append(i)
        r=[]
        #print("before", len(r))
        for m,k in enumerate(date_ob):
            
            if k>i:
                r.append(m)
                #print('after',len(r))
                if len(r) ==1:
                    #print(m,"Sell :", k, "Price:",df['c'].loc[k])
                    #print("Percentage change:",(((df['c'].loc[k]/df['c'].loc[i])-1)*100) )
                    sell.append(df['c'].loc[k])
                    sdate.append(k)
                    pct_change=((df['c'].loc[k]/df['c'].loc[i])-1)*100
                    pct.append(((df['c'].loc[k]/df['c'].loc[i])-1)*100)
                elif len(r)>1:
                    pass
    diff =len(buy)-len(sell)
    # Checking for final transaction, make sure it is sell, as we want to close the position
    if len(buy) == len(sell):
        #print("Yes \n", "Buy:", len(buy), "Sell:", len(sell))
        #print("Position closed")
        pass
    
    elif len(buy) != len(sell):
        #print("No, \n", "Buy:{}, Sell:{}, Closing the position".format(len(buy),len(sell)))
        del(buy[-diff:])
        del(bdate[-diff:])
        #print("Position closed")
    dct = {
    "Buy_Date": bdate,
    "Buy": buy,
    "Sell": sell,
    "Sell_Date": sdate,
    "Pct_change": pct }   
    trans = pd.DataFrame(dct)
    wr = []
    for i in range(len(trans)):
        if trans["Pct_change"][i] > 0:
            wr.append(1)
        else:
            wr.append(0)
    trans["wr"] = pd.Series(wr)
    
    Name_stock = []
    for s in range(len(trans)):
        Name_stock.append(symbol)
        
    #X_train, y_train = X['01-01-2016': '01-01-2021'], y['01-01-2016': '01-01-2021']
    trans['Stock'] = pd.Series(Name_stock)
    #trans['01-01-2020': '01-01-2021']
    
    trans.set_index('Buy_Date', inplace=True)
    
    trans=trans['01-01-2018':'01-01-2019']
    print(trans)
    net_pct=round(sum(trans['Pct_change']),4)
    wrate = round(len(trans[trans["wr"]==1])/len(trans["wr"])*100, 4)
    print('Stock :{}'.format(symbol))
    print('When Overbought cross>{}.'.format(ob_cross))
    print('When Oversold cross<{}.'.format(os_cross))
    print("Winning rate at {}% for {} trades and winning percentage {}%.".format(wrate, len(trans["wr"]),net_pct))
    risk = round((100-wrate),2)
    print("While, using this strategy, we are exposed to {}% of lossing risk for {} trades.".format(risk, len(trans["wr"])))
    print("----------------DONE {}------------------".format(symbol))
    

print("Finish")


----------------PROCESSING D&O------------------
              Buy   Sell  Sell_Date  Pct_change  wr Stock
Buy_Date                                                 
2018-05-04  0.630  0.645 2018-05-17    2.380952   1   D&O
2018-07-10  0.710  0.810 2018-09-24   14.084507   1   D&O
2018-07-13  0.685  0.810 2018-09-24   18.248175   1   D&O
2018-07-20  0.700  0.810 2018-09-24   15.714286   1   D&O
2018-08-28  0.740  0.810 2018-09-24    9.459459   1   D&O
2018-08-30  0.740  0.810 2018-09-24    9.459459   1   D&O
2018-09-04  0.730  0.810 2018-09-24   10.958904   1   D&O
2018-09-07  0.730  0.810 2018-09-24   10.958904   1   D&O
2018-09-13  0.725  0.810 2018-09-24   11.724138   1   D&O
2018-10-16  0.895  0.905 2018-11-08    1.117318   1   D&O
2018-10-25  0.820  0.905 2018-11-08   10.365854   1   D&O
Stock :D&O
When Overbought cross>85.
When Oversold cross<35.
Winning rate at 100.0% for 11 trades and winning percentage 114.472%.
While, using this strategy, we are exposed to 0.0% of lossing risk

In [31]:
total_wr_18 = 100 + 42.86 + 80 + 40 + 75 + 80 + 25 + 0 + 0 + 50 + 50 + 100
total_wr_18

642.86

In [32]:
round(total_wr_18/12,2)

53.57

In [34]:
total_wp_18 = 114.472 + 7.7248 + 50.2231 + (-0.5069) + 51.3555 + 40.2638 + 10.77 + (-22.1757) + (-42.9878) + (-10.7515)+ (-16.4474) + 17.662 
total_wp_18

199.60189999999997

In [35]:
round(total_wp_18/12,2)

16.63